## Pandas dataframes

Pandas dataframes are a very convenient way to interact with low-dimensional structured data. The basic dataframe object acts very similarly to an Excel file, but data can be manipulated with Python rather than clumsy Excel functions.

In [ ]:
%matplotlib inline
import pandas as pd

#Look at csv file 
df = pd.read_csv('/Users/robertdickson/Documents/datasets/oceanic_data-brewer.csv',skiprows=32)
#df['DATE']
#df

Pandas dataframes are a cross between dictionaries and numpy arrays. Unlike arrays, they are allowed to hold multiple types, and they index columns and rows based on "keys" rather than numbers:

In [ ]:
#print(df.index) #row names
print(df.columns) # column names
#print(type(df.values)) #all data
#print(df.dtypes)

It is easy to view and summarize data with the dataframe object:

In [ ]:
df.head(5)
#df.tail(10)
#df

Data can be accessed in several ways:

* Columns can be accessed directly with keys. 
* The `loc` method enables numpy-like indexing, fancy indexing, and slicing. 
* The `iloc` method is similar to `loc`, but indexes by position (rather than key)

These methods return `pandas.Series` objects, that are basically 1D dataframes.

In [ ]:
#df['LATITUDE']
#df.loc[1,'LATITUDE']
df.loc[6]
#df.loc[:,['LATITUDE','LONGITUDE']]
#df.iloc[:,7]

Columns can be deleted in three ways:

* del : delete the Series from the dataframe
* pop() : delete the Series and return the Series
* drop(labels, axis) : return a new dataframe with Series removed (do not modify original df)

Rows must be "dropped".

**Pay attention to whether operations are "in place" or not**. Many `pandas` operations are *not* "in place" by default. This means that they return a copy of the dataframe with modifications, rather than modifying the original dataframe object. This can be very confusing.

In [ ]:
#Delete column
times = df.pop('TIME')
print(times)
del df['EXPOCODE']
dfnew = df.drop('CASTNO', axis=1)

#Delete row
df2 = df.drop(df.index[0])
df2 = df2.drop(df.index[-1])
df2.head(3)

Let's simplify this data into only the columns that we care about:

In [ ]:
interesting = ['DATE','LATITUDE','LONGITUDE','DEPTH','SALNTY','OXYGEN','SILCAT','NITRAT','PHSPHT', 'ALKALI']
df = df[interesting]
df = df.drop(df.index[0])
df = df.drop(df.index[-1])
df

It looks like there are some artifacts since a concentration of -999.0 does not make sense. We can get rid of these using "boolean indexing":

In [ ]:
df.dtypes
df = df.astype('float')
df.dtypes
conc = ['OXYGEN','SILCAT','NITRAT','PHSPHT', 'ALKALI']
for var in conc:
    df = df[df[var] >= 0] #only take rows where concentration is >=0

df = df.reset_index() #<- THIS IS IMPORTANT!!!
df

This could also be achieved with OpenRefine. There is often no right way to do things in data science, so pick whatever feels easiest or most intuitive.

Note that the dates were not parsed correctly. We can fix this with a loop and assign the proper dates to the DATE column:

In [ ]:
def convert_DATE(date_col):
    dates = []
    for dt in date_col:
        dt = str(int(dt))
        y,m,d = dt[:4], dt[4:6], dt[6:]
        dates.append('-'.join([y,m,d]))
    print(len(dates))
    dates = pd.Series(dates)
    dates = pd.to_datetime(dates,yearfirst=True)
    return dates

dates = convert_DATE(df['DATE'])
df['DATE'] = dates # [(0, date0), (1, date1) ... ]

In [ ]:
df.head(10)

Pandas dataframes have some handy plotting features built in:

In [ ]:
print(conc)
df[conc].plot()
df_sorted = df.sort_values('PHSPHT')
df_sorted.plot(x='PHSPHT',y='SILCAT')
df_sorted.hist('SILCAT')

Pandas allows easily indexing by different columns:

In [ ]:
df2 = df.set_index('DATE')
df2['1981-06-03':'1981-09-07']

## Pandas multi-indexing

Pandas multi-indexing allows multi-dimensional `DataFrame` objects that act like different sheets/files in Excel. This is very useful for creating "mini databases" that allow organization of complex data with multiple dimensions.

Let's consider another dataset similar to the one we just worked with:

In [ ]:
new_df = pd.read_csv('datasets/oceanic_data-morliere.csv',skiprows=35)
#quickly clean up the data:
new_df = new_df[interesting]
new_df = new_df.drop(0)
new_df = new_df.astype('float')
for var in conc:
    new_df = new_df[new_df[var] > 0]
new_df = new_df.reset_index()
dates = convert_DATE(new_df['DATE'])
new_df['DATE'] = dates
new_df = new_df.set_index('DATE')
new_df

Now we might want to create a DateFrame that contains both sets of data, but index them by the scientist that collected the data (Brewer and Morliere). This can be achieved with the Multi Index functionality:

In [ ]:
brewer = df2
morliere = new_df
full = pd.concat({'Brewer':brewer, 'Morliere':morliere})
#print(full.columns)
full = full.T
#print(full.columns)
x = full['Morliere'].T['1993-02'] #<- get all measurements by Brewer in May 1981
print(x.columns)
x.head(10)

It is easy to convert from a `pandas` dataframe back to a `numpy` array (assuming all values are numerical):

In [ ]:
x.values